In [ ]:
from dataclasses import dataclass
from collections import deque

@dataclass
class Node:
    name : str
    offset : list
    channels : dict
    child : list
    parent : str
    first_in : bool
    
file_path = "walk60.bvh"
bvh_data = []
with open(file_path, 'r') as f:
    for line in f:
        bvh_data.append(line.split())

num_frames = 0
frame_time = 0
frames = []
joint_List = []
info_List = []
for line in bvh_data:
    this_line = line
    if(this_line[0] == "Hierarchy"): continue
    if(this_line[0] == "MOTION"): continue
    if(this_line[0] == "Frames:"):
        num_frames = int(this_line[1])
        continue
    if(this_line[0] == "Frame"):
        frame_time = float(this_line[2])
        continue
    if(this_line[0] == "{" or
       this_line[0] == "ROOT" or
       this_line[0] == "}" or
       this_line[0] == "End" or
       this_line[0] == "JOINT" or
       this_line[0] == "OFFSET" or
       this_line[0] == "CHANNELS"):
        info_List.append(this_line)
        continue
    frames.append(this_line)
    
# # 处理栈中数据
# def Process_data(stack):
#     some_data = []
#     while(stack):
#         info_pop = stack.pop()
#         if(type(info_pop) != Node and info_pop[0] == "{"):
#             break
#         elif(type(info_pop) != Node and info_pop[0] == "}"):
#             continue
#         else:
#             some_data.append(info_pop)
#     print(some_data)
#     now_node = Node(
#         name = None,
#         offset = [],
#         channels = {},
#         child = [],
#         parent = None,
#         first_in = True
#     )
#     last_node = None
#     for i in some_data:
#         # if(type(i) != Node and i[0] == "OFFSET"):
#         #     now_node.offset = [float(i[1]), float(i[2]), float(i[3])]
#         # elif(type(i) != Node  and i[0] == "CHANNELS"):
#         #     now_node.channels = {
#         #         "num_channels" : int(i[1]),
#         #         "channels" : [i[1:]]
#         #     }
#         # elif(type(i) != Node and i[0] == "JOINT"):
#         #     now_node.name = i[1]
#         # elif(type(i) == Node):
#         #     last_node = i
#         #     i += 1
#         if(type(i) == Node):
#             last_node = i
#             continue
#         if(type(i) != Node and i[0] == "OFFSET"):
#             now_node.offset = [float(i[1]), float(i[2]), float(i[3])]
#         elif(type(i) != Node and i[0] == "CHANNELS"):
#             now_node.channels = {
#                 "num_channels" : int(i[1]),
#                 "channels" : [i[1:]]}
#         elif(type(i) != Node and i[0] == "End" or i[0] == "Joint"):
#             if(i[0] == "End") : last_node.name = i[0] + i[1]
#             if(i[0] == "Joint") : last_node.name = i[1]
#             last_node.parent = now_node.name
#             now_node.child.append(last_node)
#             last_node.first_in = False
#             joint_List.append(last_node)
            
#         print(now_node)
#         print(last_node)
#     return now_node
        
    
            
len_info = len(info_List)
node_list = []
stack = deque()
# for i in range(len_info):
#     if(info_List[i][0] == "{"):
#         stack.append(info_List[i])
#         while(info_List[i][0] != "}"):
#             i += 1
#             stack.append(info_List[i])
#     elif(info_List[i][0] == "}"):
#         stack.pop()
#         print(stack.pop())
for i in range(len_info):
    if info_List[i][0] == "ROOT":
        root = Node(
             name = info_List[i][0] + info_List[i][1],
                offset = [],
                channels = {},
                child = [],
                parent = None,
                first_in = True)
        node_list.append(root)
        joint_List.append(root)
        continue
    if info_List[i][0] == "{" or info_List[i][0] == "OFFSET" or \
        info_List[i][0] == "CHANNELS" or info_List[i][0] == "End" or \
        info_List[i][0] == "JOINT":
            stack.append(info_List[i])
            continue
    if info_List[i][0] == "}":
        stack.append(info_List[i])

 
       
    
        


In [ ]:
info_List

In [ ]:
node_list

In [ ]:
class Node:
    def __init__(self, name):
        self.name = name
        self.offset = None
        self.channels = []
        self.children = []

    def __repr__(self):
        return f"Node({self.name})"


def build_skeleton(bvh_data):
    stack = []
    root = None
    current_node = None

    for line in bvh_data:
        if line[0] == 'ROOT' or line[0] == 'JOINT':
            # 创建新节点
            node = Node(line[1])
            if current_node is not None:
                # 添加为当前节点的子节点
                current_node.children.append(node)
                stack.append(current_node)
            else:
                root = node
            current_node = node
        elif line[0] == 'OFFSET':
            # 设置偏移量
            current_node.offset = tuple(map(float, line[1:]))
        elif line[0] == 'CHANNELS':
            # 设置通道信息
            current_node.channels = line[2:]  # 跳过通道数量
        elif line[0] == '}':
            # 结束当前节点的定义，返回到父节点
            if stack:
                current_node = stack.pop()
        elif line[0] == 'End' and line[1] == 'Site':
            # 处理末端节点
            end_site_node = Node('End Site')
            current_node.children.append(end_site_node)
            stack.append(current_node)
            current_node = end_site_node

    return root

# 解析示例数据
bvh_data = [['ROOT', 'Hips'], ['{'], ['OFFSET', '0.00', '0.00', '0.00'], ['CHANNELS', '6', 'Xposition', 'Yposition', 'Zposition', 'Xrotation', 'Yrotation', 'Zrotation']]
skeleton = build_skeleton(info_List)
print(skeleton.children)


In [ ]:
from dataclasses import dataclass
from collections import deque

@dataclass
class Node:
    name : str
    offset : list
    channels : dict
    child : list
    parent : str
    ID : int
    type : str
    
file_path = "walk60.bvh"
bvh_data = []
with open(file_path, 'r') as f:
    for line in f:
        bvh_data.append(line.split())

num_frames = 0
frame_time = 0
frames = []
joint_List = []
info_List = []
for line in bvh_data:
    this_line = line
    if(this_line[0] == "Hierarchy"): continue
    if(this_line[0] == "MOTION"): continue
    if(this_line[0] == "Frames:"):
        num_frames = int(this_line[1])
        continue
    if(this_line[0] == "Frame"):
        frame_time = float(this_line[2])
        continue
    if(this_line[0] == "{" or
       this_line[0] == "ROOT" or
       this_line[0] == "}" or
       this_line[0] == "End" or
       this_line[0] == "JOINT" or
       this_line[0] == "OFFSET" or
       this_line[0] == "CHANNELS"):
        info_List.append(this_line)
        continue
    frames.append(this_line)
    
offset_stack = deque()
channel_stack = deque()
node_stack = deque()
joint_List = []
node_id = 0

for info in info_List:
    if(info[0] == "ROOT"):
        node_stack.append(Node(
            name = "RootJoint",
            offset = None,
            channels = None,
            child = [],
            parent = None,
            ID = node_id,
            type = "root"
        ))
        node_id += 1
        continue
    elif info[0] == "{": continue
    elif info[0] == "OFFSET":
        offset_stack.append(list(map(float, info[1:])))
        continue
    elif info[0] == "CHANNELS":
        channel_stack.append({
            "channel_num" : int(info[1]),
            "channels" : info[2:]
        })
    elif info[0] == "JOINT":
        node_stack.append(Node(
            name = info[1],
            offset = None,
            channels = None,
            child = [],
            parent = None,
            ID = node_id,
            type = "joint"
        ))
        node_id += 1
        continue
    elif info[0] == "End":
        node_stack.append(Node(
            name = info[0] + info[1],
            offset = None,
            channels = None,
            parent = None,
            child = None,
            type = "End",
            ID = node_id
        ))
        node_id += 1
        continue
    elif info[0] == "}":
        pop_node = node_stack.pop()
        if pop_node.type == "End":
            top_node = node_stack[-1]
            pop_node.name = top_node.name + "_end"
            pop_node.parent = top_node.name
            pop_node.child = None
            offset_top = offset_stack.pop()
            pop_node.offset = offset_top
            pop_node.channels = None
            top_node.child.append(pop_node)
            joint_List.append(pop_node)
        elif pop_node.type == "joint":
            top_node = node_stack[-1]
            pop_offset = offset_stack.pop()
            pop_channels = channel_stack.pop()
            # print(pop_channels)
            pop_node.offset = pop_offset
            pop_node.channels = pop_channels
            pop_node.parent = top_node.name
            top_node.child.append(pop_node)
            joint_List.append(pop_node)
        elif pop_node.type == "root":
            pop_offset = offset_stack.pop()
            pop_channels = channel_stack.pop()
            # print(pop_channels)
            pop_node.offset = pop_offset
            pop_node.channels = pop_channels
            pop_node.parent = None
            joint_List.append(pop_node)
        continue
               
            

In [ ]:
for node in joint_List:
    if(node.name == "RootJoint"):
        print(len(node.child))

In [ ]:
from read_bvh import Bone_Tree

bone_tree = Bone_Tree('walk60.bvh')

In [ ]:
bone_tree.all_channels

In [ ]:
bone_tree.channels

In [ ]:
bone_tree.frames[0]

In [ ]:
from read_bvh import Bone_Tree

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R
def load_motion_data(bvh_file_path):
    """part2 辅助函数，读取bvh文件"""
    with open(bvh_file_path, 'r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            if lines[i].startswith('Frame Time'):
                break
        motion_data = []
        for line in lines[i+1:]:
            data = [float(x) for x in line.split()]
            if len(data) == 0:
                break
            motion_data.append(np.array(data).reshape(1,-1))
        motion_data = np.concatenate(motion_data, axis=0)
    return motion_data

In [ ]:
motion_data = load_motion_data(bvh_file_path= 'walk60.bvh')

In [ ]:
motion_data

In [ ]:
print(motion_data.shape)

In [ ]:
motion_data_my = Bone_Tree('walk60.bvh').frames

In [ ]:
motion_data_my

In [ ]:
from read_bvh import Bone_Tree
tree = Bone_Tree(bvh_file_path= 'walk60.bvh')

In [ ]:
tree.sorted_joint_list

In [ ]:
name_list = []
id_list = []
parent_list = []
offset_list = []
for joint in tree.sorted_joint_list:
    name_list.append(joint.name)
    id_list.append(joint.ID)
    parent_list.append(joint.parent["parent_id"])
    offset_list.append(joint.offset)
print(name_list)
print(id_list)
print(parent_list)
print(len(offset_list))
print(tree.node_count)

In [ ]:
tree.joint_List

In [ ]:
from read_bvh import Bone_Tree
tree = Bone_Tree(bvh_file_path= 'walk60.bvh')

In [ ]:
print(tree.joint_name)
print(tree.)
print(tree.joint_parent)

In [ ]:
from read_bvh import Bone_Tree
from dataclasses import dataclass
from collections import deque
import numpy as np
from scipy.spatial.transform import Rotation as R

bone_tree = Bone_Tree("walk60.bvh")

In [ ]:
joint_List = bone_tree.sorted_joint_list
joint_name = bone_tree.joint_name
joint_parent = bone_tree.joint_parent
joint_offset = bone_tree.joint_offset
motion_data = bone_tree.frames


joint_List

In [ ]:
def find_parent_rotation(Q, parent_index):
    for joint in Q:
        if joint['id'] == parent_index:
            return joint['rotation']
        
def find_parent_location(Q, parent_index):
    for joint in Q:
        if joint['id'] == parent_index:
            return joint['location']

def forward_kinematics(
        joint_list: list,
        motion_data: list,
    ) -> np.array:
        # Q4 = Q3R4 = Q2R3R4 = Q1R2R3R4 = Q0R1R2R3R4 = R0R1R2R3R4
        # DP 累推
        # return : joint_position : shape -> num_frames * num_joint * 3
        # return : joint_rotation : shape -> num_frames * num_joint * 4 (quaternion)
        joint_all_frames_info_rotation_local = []
        for frame in motion_data:
            # Q : this_frame joint_rotation -> shape : num_joint * 4
            Q = []
            # Only root joint has translation
            T0 = None
            # joint_list is sorted by ID : the sequence follow the bvh hierarchy
            for joint in joint_list:
                if joint.type == "root":
                    # root channels is 6 : first 3 is translate , second 3 is rotation
                    translate = frame[:3]
                    rotation = frame[3:6]
                    # translate_euler = R.from_euler('XYZ', translate, degrees=True)
                    T0 = np.array(translate)
                    rotation_euler = R.from_euler("XYZ", rotation, degrees=True)
                    rotation_quaternion = rotation_euler.as_quat()
                    # root's location is (0,0,0)
                    Q.append(
                        {
                            "id": 0,
                            "rotation" : rotation_quaternion,
                            "location" : np.array([0, 0, 0]),
                        }
                    )
                elif joint.type == "joint":
                    joint_index = joint.ID - 1
                    parent_index = joint.parent["parent_id"]
                    joint_rotation = frame[
                       joint.channel_index : joint.channel_index + 3
                    ]
                    #print(joint.ID)
                    #print(joint_index)
                    joint_rotation = np.array(list(map(float, joint_rotation)))
                    # parent_roation is quaternion
                    parent_rotation = find_parent_rotation(Q, parent_index)
                    #print(joint_rotation)
                    #print(type(joint_rotation))
                    #print(R.from_euler("xyz", joint_rotation, degrees=True))
                    #print(R.from_euler("xyz", joint_rotation, degrees=True).as_matrix())
                    this_joint_rotation = R.from_matrix(
                        R.from_quat(parent_rotation).as_matrix()
                        @ R.from_euler("xyz", joint_rotation, degrees=True).as_matrix()
                    ).as_quat()
                    # print(this_joint_rotation)
                    # print("--------------")
                    # parent_location is np.array -> 3 dim vec
                    parent_location = find_parent_location(Q, parent_index)
                    # joint_offset is np.array -> 3 dim vec
                    joint_offset = joint.offset
                    # ID == 1 -> need translate
                    if joint.ID == 1:
                        T0 = list(map(float, T0))
                        this_joint_location = (
                            parent_location
                            + R.from_quat(parent_rotation).apply(np.array(joint_offset))
                            + T0
                        )
                    else:
                        this_joint_location = (
                            parent_location + R.from_quat(parent_rotation).apply(np.array(joint_offset))
                        )
                    Q.append(
                        {
                            'id' : joint.ID,
                            "rotation" : this_joint_rotation,
                            "location" : this_joint_location,
                        }
                    )
                elif joint.type == "End":
                    parent_index = joint.parent["parent_id"]
                    parent_rotation = find_parent_rotation(Q, parent_index)
                    parent_location = find_parent_location(Q, parent_index)
                    this_offset = joint.offset
                    this_joint_location = parent_location + R.from_quat(parent_rotation).apply(np.array(this_offset))
                    Q.append(
                        {
                            "id" : joint.ID,
                            "rotation" : None,
                            "location" : this_joint_location
                        }
                    )
            joint_all_frames_info_rotation_local.append(Q)
        return joint_all_frames_info_rotation_local
                
                    

In [ ]:
joint_all_position = forward_kinematics(
    joint_List,
    motion_data = motion_data)
print(np.array(joint_all_position).shape)

In [ ]:
print(joint_all_position[0])

In [ ]:
from read_bvh import Bone_Tree
from dataclasses import dataclass
from collections import deque
import numpy as np
from scipy.spatial.transform import Rotation as R

bone_tree = Bone_Tree("walk60.bvh")

In [ ]:
bone_tree.all_frame_location

In [ ]:
bone_tree.all_frame_rotation

In [ ]:
from read_bvh import Bone_Tree
from scipy.spatial.transform import Rotation as R
import numpy as np

In [ ]:
bone_tree_A_pos = Bone_Tree("A_pose_run.bvh")
bone_tree_T_pos = Bone_Tree("walk60.bvh")

In [ ]:
bone_tree_A_pos.joint_name_without_end

In [ ]:
bone_tree_A_pos.joint_parent

In [ ]:
bone_tree_T_pos.joint_parent

In [ ]:
bone_tree_A_pos_joint_name_without_end = bone_tree_A_pos.joint_name_without_end
bone_tree_T_pos_joint_name_without_end = bone_tree_T_pos.joint_name_without_end
A_pos_frame = bone_tree_A_pos.frames
A_pos_processed_frame = bone_tree_A_pos.process_A_pos_frame(A_pos_frame = A_pos_frame,
                                                            T_pos_joint_name_without_end = bone_tree_T_pos_joint_name_without_end,
                                                            A_pos_joint_name_without_end = bone_tree_A_pos_joint_name_without_end)

In [ ]:
from scipy.spatial.transform import Rotation as R
# first step -> calculate all Q-matrices
# Q-matrix is transfrom T-pos to A-pos
def calculate_Q_matrix(A_pos_frame : list, T_pos_frame : list, joint_name : list) -> list:
    Q_matrix_all_frame = [] # -> dict : {joint_name : Q_matrix} -> shape:(num_frames, num_joints)
    for A_pos_frame_this, T_pos_frame_this in zip(A_pos_frame, T_pos_frame):
        Q_matrix_this_frame = []
        print(joint_name)
        for idx, joint in enumerate(joint_name):
            A_pos_rotation = A_pos_frame_this[3 + idx * 3 : 6 + idx * 3]
            # print("A_pos_rotation : ", A_pos_rotation)
            T_pos_rotation = T_pos_frame_this[3 + idx * 3 : 6 + idx * 3]
            # print("T_pos_rotation : ", T_pos_rotation)
            A_rotation_matrix = R.from_euler("XYZ", A_pos_rotation, degrees = True)
            print("A_rotation_matrix : ", A_rotation_matrix.as_matrix())
            T_rotation_matrix = R.from_euler("XYZ", T_pos_rotation, degrees = True)
            print("T_rotation_matrix : ", T_rotation_matrix.as_matrix())
            # target_matrix * T_rotation_matrix = A_rotation_matrix
            # so -> target_matrix = A_rotation_matrix * T_rotation_matrix.inv()
            Q_this_joint = A_rotation_matrix * T_rotation_matrix.inv()
            Q_matrix_this_frame.append(
                {
                    "joint_name" : joint,
                    "Q_matrix" : Q_this_joint
                }
            )
        Q_matrix_all_frame.append(Q_matrix_this_frame)
    return Q_matrix_all_frame

In [ ]:
Q_matrix = calculate_Q_matrix(A_pos_frame = A_pos_processed_frame, 
                              T_pos_frame = bone_tree_T_pos.frames,
                              joint_name = bone_tree_A_pos.joint_name_without_end)
print(Q_matrix)

In [ ]:
Q_matrix

In [ ]:
# ok -> all_Q_has_been_calculated
Q_matrix[0][0]['Q_matrix'].as_matrix() 
print((Q_matrix[0]))


In [ ]:
bone_tree_A_pos.sorted_joint_list

In [ ]:
Q_matrix[1]

In [ ]:
def find_node(node_list : list, joint_name : str):
    for node in node_list:
        if(node.name == joint_name):
            return node

def find_Q(Q_matrix : list , joint_name : str):
    for Q in Q_matrix:
        if(Q['joint_name'] == joint_name):
            return Q['Q_matrix']

# for root node -> Rotation_in_A_pos = Rotation_in_T_pos * Q(from_T_pos_to_A_pos)^T
# for none root node -> Rotation_in_A_pos = Q(parent_from_T_pos_to_A_pos) * Rotation_in_T_pos * Q(this_joint_from_T_pos_to_A_pos)

def motion_retarget(Q_matrix : list, T_pos_all_frames : list, node_list_in_A_pos : list, node_list_in_T_pos : list) -> list:
    # input : rotation_in_A_pos_all_frames,
    #         tranformation_matrix_from_T_pos_to_A_pos
    #         rotation_in_T_pos_all_frames
    # output : retargeting_motion_from_A_pos_to_T_pos_all_frames
    all_frame_new_rotation_in_T_pos = []
    for idx, T_pos_frame in enumerate(T_pos_all_frames):
        this_T_pos_frame = T_pos_frame
        # print(this_T_pos_frame)
        # print(idx)
        # print(Q_matrix)
        this_frame_all_Q_matrix = Q_matrix[int(idx)]
        # print(this_frame_all_Q_matrix)
        this_frame_new_rotation_in_T_pos = []
        for idx_joint, one_Q_matrix in enumerate(this_frame_all_Q_matrix):
            joint_name = one_Q_matrix['joint_name']
            Q_matrix_this_joint = one_Q_matrix['Q_matrix']
            this_joint_rotation_in_T_pos = this_T_pos_frame[3 + 3 * idx_joint : 6 + 3 * idx_joint]
            joint_node_in_A_pos = find_node(node_list_in_A_pos, joint_name)
            joint_parent_name = joint_node_in_A_pos.parent['parent_name'] 
            parent_Q_matrix = find_Q(this_frame_all_Q_matrix, joint_parent_name)
            if(joint_node_in_A_pos.type == 'root'):
                # this is a root node
                # print("this_joint_rotation_in_T_pos: ", this_joint_rotation_in_T_pos)
                # print("Q_matrix_this_joint: ", Q_matrix_this_joint.as_matrix())
                # add T_pos translation to the new_frame_info
                this_frame_new_rotation_in_T_pos.append(this_T_pos_frame[0 : 3])
                B_rotation = R.from_euler("XYZ", this_joint_rotation_in_T_pos, degrees = True).as_matrix() * Q_matrix_this_joint.as_matrix().transpose()
                this_frame_new_rotation_in_T_pos.append(R.from_matrix(B_rotation).as_euler("XYZ", degrees = True).tolist())
                continue
            else:
                B_rotation = parent_Q_matrix.as_matrix() * R.from_euler("XYZ", this_joint_rotation_in_T_pos, degrees = True).as_matrix() * Q_matrix_this_joint.as_matrix().transpose()
                this_frame_new_rotation_in_T_pos.append(R.from_matrix(B_rotation).as_euler("XYZ", degrees = True).tolist())
        all_frame_new_rotation_in_T_pos.append(np.array(this_frame_new_rotation_in_T_pos).reshape(-1, len(this_T_pos_frame)).tolist()) 
    return np.array(all_frame_new_rotation_in_T_pos).squeeze().tolist()
                     

In [ ]:
new_frame = motion_retarget(
    Q_matrix,
    T_pos_all_frames = bone_tree_T_pos.frames,
    node_list_in_A_pos = bone_tree_A_pos.sorted_joint_list,
    node_list_in_T_pos = bone_tree_T_pos.sorted_joint_list
)

In [ ]:
new_frame

In [5]:
from read_bvh import Bone_Tree
import numpy as np

bone_tree_T_pos = Bone_Tree("walk60.bvh")

In [27]:
a_path_b = bone_tree_T_pos.get_path_from_Ajoint_To_Bjoint("lToeJoint_end", "RootJoint")

In [29]:
joint_offsets = bone_tree_T_pos.joint_offset
# len(joint_offsets)
root_joint_name = a_path_b[1][-1]
end_joint_name = a_path_b[1][0]
# print(root_joint_name, end_joint_name)
root_joint_name_index = bone_tree_T_pos.joint_name.index(root_joint_name)
end_joint_name_index = bone_tree_T_pos.joint_name.index(end_joint_name)
print(root_joint_name_index, end_joint_name_index)

0 5


In [30]:
root_joint_offset = joint_offsets[root_joint_name_index]
root_joint_offset

array([0., 0., 0.])

In [31]:
end_joint_offset = joint_offsets[end_joint_name_index]
end_joint_offset

array([0.01 , 0.002, 0.06 ])

In [26]:
a_path_b = bone_tree_T_pos.get_path_from_Ajoint_To_Bjoint("lToeJoint_end", "RootJoint")

In [6]:
a_path = bone_tree_T_pos.get_path_from_Ajoint_To_Bjoint("lToeJoint_end", "lWrist_end")

In [7]:
a_path

([[5, 4, 3, 2, 1, 0], [12, 11, 10, 9, 8, 7, 6, 0]],
 [['lToeJoint_end', 'lToeJoint', 'lAnkle', 'lKnee', 'lHip', 'RootJoint'],
  ['lWrist_end',
   'lWrist',
   'lElbow',
   'lShoulder',
   'lTorso_Clavicle',
   'lowerback_torso',
   'pelvis_lowerback',
   'RootJoint']])

In [8]:
a_path_b = bone_tree_T_pos.get_path_from_Ajoint_To_Bjoint("lToeJoint_end", "RootJoint")

In [9]:
a_path_b

([5, 4, 3, 2, 1, 0],
 ['lToeJoint_end', 'lToeJoint', 'lAnkle', 'lKnee', 'lHip', 'RootJoint'])

In [10]:
primary_positions, primary_rotations = bone_tree_T_pos.get_bone_tree_primary_positions_rotations_from_AJoint_To_BJoint(
    path_info = a_path_b
 )

In [11]:
primary_positions, primary_rotations

([array([ 0.02321566, -0.01050921,  0.21952044]),
  array([ 0.01563993, -0.01153512,  0.159142  ]),
  array([0.02119855, 0.05747961, 0.03828643]),
  array([0.0740268 , 0.40404407, 0.20917308]),
  array([0.09666573, 0.80309773, 0.30051669]),
  array([0., 0., 0.])],
 [array([ 0.00652976, -0.02040147, -0.01068351,  0.99971346]),
  array([ 0.00652976, -0.02040147, -0.01068351,  0.99971346]),
  array([ 0.07547421, -0.02394533,  0.00391293,  0.99685252]),
  array([ 0.22701368,  0.02261524, -0.0644312 ,  0.9714947 ]),
  array([ 0.1130767 ,  0.03588904, -0.02372451,  0.99265441]),
  array([ 0.017026  ,  0.06643844, -0.00578745,  0.99762847])])

In [12]:
primary_positions.reverse()

In [13]:
print(primary_positions)

[array([0., 0., 0.]), array([0.09666573, 0.80309773, 0.30051669]), array([0.0740268 , 0.40404407, 0.20917308]), array([0.02119855, 0.05747961, 0.03828643]), array([ 0.01563993, -0.01153512,  0.159142  ]), array([ 0.02321566, -0.01050921,  0.21952044])]


In [14]:
import torch as torch
import numpy as np

In [15]:
aa = np.array([x.tolist() for x in primary_rotations])

In [16]:
aa

array([[ 0.00652976, -0.02040147, -0.01068351,  0.99971346],
       [ 0.00652976, -0.02040147, -0.01068351,  0.99971346],
       [ 0.07547421, -0.02394533,  0.00391293,  0.99685252],
       [ 0.22701368,  0.02261524, -0.0644312 ,  0.9714947 ],
       [ 0.1130767 ,  0.03588904, -0.02372451,  0.99265441],
       [ 0.017026  ,  0.06643844, -0.00578745,  0.99762847]])

In [17]:
a = torch.from_numpy(aa)


In [18]:
a.requires_grad_(True)

tensor([[ 0.0065, -0.0204, -0.0107,  0.9997],
        [ 0.0065, -0.0204, -0.0107,  0.9997],
        [ 0.0755, -0.0239,  0.0039,  0.9969],
        [ 0.2270,  0.0226, -0.0644,  0.9715],
        [ 0.1131,  0.0359, -0.0237,  0.9927],
        [ 0.0170,  0.0664, -0.0058,  0.9976]], dtype=torch.float64,
       requires_grad=True)

In [20]:
print(a.grad)

None


In [22]:
len(a)

6

In [37]:
a[0].tolist()

[0.006529755862589982,
 -0.020401466056178146,
 -0.010683507664969403,
 0.9997134615154015]

In [50]:
from scipy.spatial.transform import Rotation as R
Q = []
Q.append(R.from_euler('xyz', [0, 0, 0], degrees=True).as_matrix())
for i in range(len(aa)):
     #Q.append(R.from_euler("XYZ", degrees = True, seq = a[i].tolist()))
     # print(a[i].tolist())
     # print(R.from_quat(a[i].tolist()).as_matrix())
     # print(Q[-1])
     Q.append(
        R.from_quat(a[i].tolist()).as_matrix() @ Q[-1]
    )
Q[1:]

[array([[ 0.99893929,  0.02109446, -0.04093076],
        [-0.02162733,  0.99968645, -0.01261985],
        [ 0.04065172,  0.01349169,  0.99908228]]),
 array([[ 0.99575957,  0.0416077 , -0.08204675],
        [-0.04373795,  0.99874652, -0.02433894],
        [ 0.08093122,  0.02782429,  0.99633124]]),
 array([[ 0.99127064,  0.02884539, -0.1286486 ],
        [-0.05126251,  0.98331967, -0.17451238],
        [ 0.12146882,  0.17958385,  0.97621461]]),
 array([[ 0.97686661,  0.16441166, -0.1367495 ],
        [-0.21340333,  0.79075416, -0.57373066],
        [ 0.01380723,  0.58964113,  0.80754734]]),
 array([[ 0.96237672,  0.24631481, -0.11471731],
        [-0.24891114,  0.62985895, -0.73574516],
        [-0.10896921,  0.73661844,  0.66747209]]),
 array([[ 0.93595517,  0.35032432, -0.03550765],
        [-0.2539003 ,  0.60157421, -0.75739231],
        [-0.24397245,  0.71790065,  0.65199394]])]

In [51]:
def forward_kinematics(root_joint_pos : list, all_joint_offset : list, rotation_param_matrix : torch.Tensor)-> list:
    # do forward kinematics from a root joint to a end joint
    # return -> end joint position 
    
    # first calculate all needed rotations
    Q = []
    Q.append(
        R.from_euler("XYZ",[0,0,0], degrees = True).as_matrix()
    )
    for i in range(len(rotation_param_matrix)):
        Q.append(
            R.from_quat(rotation_param_matrix[i].tolist()).as_matrix() @ Q[-1]
        )
    # exclude the identity matrix
    Q = Q[1:]
    
    # second calculate the end joint position
    '''
    if the joint nums is 5, the final position is calculated by :
    X = p0 + R0l0 + R0R1l1 + R0R1R2l2 + R0R1R2R3l3 + R0R1R2R3R4l4
    Here p0 is root_joint_pos
         R0 - R0R1R2R3R4 is calculated in Q
         l0 - l4 - x is all offset from root joint to end joint
    '''
    final_position = np.array(root_joint_pos)
    for i in range(len(Q)):
        final_position += np.array(
            Q[i] @ all_joint_offset[i]
        )
    return final_position.tolist()

In [24]:
from read_bvh import Bone_Tree
from torch import torch
import numpy as np
from scipy.spatial.transform import Rotation as R

def get_joint_offset_along_path(bone_tree : Bone_Tree, path_name : list):
    path_offset_along_path = []
    
    '''
    path_name is like : ['lToeJoint_end', 'lToeJoint', 'lAnkle', 'lKnee', 'lHip', 'RootJoint']
              is from end to root joint
    '''
    all_joint_name_index = [
        bone_tree.joint_name.index(joint_name)
        for joint_name in path_name
    ]
    
    path_offset_along_path = [
        bone_tree.joint_offset[joint_name_index]
        for joint_name_index in all_joint_name_index
    ]
    # so the offset is from end to root joint
    path_offset_along_path.reverse()
    return path_offset_along_path
    
def forward_kinematics(root_joint_pos : list, all_joint_offset : list, rotation_param_matrix : torch.Tensor)-> list:
    # do forward kinematics from a root joint to a end joint
    # return -> end joint position 
    
    # first calculate all needed rotations
    Q = []
    Q.append(
        R.from_euler("XYZ",[0,0,0], degrees = True).as_matrix()
    )
    for i in range(len(rotation_param_matrix)):
        Q.append(
            R.from_quat(rotation_param_matrix[i].tolist()).as_matrix() @ Q[-1]
        )
    # exclude the identity matrix
    Q = Q[1:]
    
    # second calculate the end joint position
    '''
    if the joint nums is 5, the final position is calculated by :
    X = p0 + R0l0 + R0R1l1 + R0R1R2l2 + R0R1R2R3l3 + R0R1R2R3R4l4
    Here p0 is root_joint_pos
         R0 - R0R1R2R3R4 is calculated in Q
         l0 - l4 - x is all offset from root joint to end joint
    '''
    final_position = np.array(root_joint_pos)
    for i in range(len(Q)):
        final_position += np.array(
            Q[i] @ all_joint_offset[i]
        )
    return final_position.tolist()

def inverse_kinematics(
    bone_tree: Bone_Tree,
    path_info: list,
    # joint_positions: list,
    # joint_orientations: list,
    target_pos: list,
) -> list:
    """
    input : path_info : list -> from AJoint To BJoint path info
            joint_positions : list -> no IK joint positions
            joint_orientations : list -> no IK joint orientations
            target_pos : list -> target position
    output :
            joint_positions : list -> IK joint positions
            joint_orientations : list -> IK joint orientations
    """
    # in gradient descent -> update the rotate parameter of each joint
    # then calculate the new joint position
    # the primart position for root joint is all the same -> not change
    # every iterate -> do forward kinamatics to calculate the end joint position

    '''
    all needed info :
    thresold : 0.001
    learning_rate : 0.01
    '''
    thresold = 0.001
    learning_rate = 0.01
    epochs = 1000

    try:
        path_index = path_info[0]
        path_name = path_info[1]
    except:
        if len(path_index) == 2:
            raise ValueError("more path here not supported")
    # this only process one path case

    primary_positions, primary_rotations = (
        bone_tree.get_bone_tree_primary_positions_rotations_from_AJoint_To_BJoint(
            path_info = path_info
        )
    )
    
    # offset from root joint to end joint
    root_to_end_offset = get_joint_offset_along_path(bone_tree, path_name)

    # reverse -> from RootJoint to end joint
    primary_positions.reverse()
    primary_rotations.reverse()

    # type : np.ndarray
    primary_positions_root = primary_positions[0]

    # create a rotation parameter matrix
    rotation_matrix = torch.from_numpy(
        np.array([x.tolist() for x in primary_rotations])
    ).requires_grad_(True)
    """
    torch array like :
    tensor([[ 0.0065, -0.0204, -0.0107,  0.9997],
        [ 0.0065, -0.0204, -0.0107,  0.9997],
        [ 0.0755, -0.0239,  0.0039,  0.9969],
        [ 0.2270,  0.0226, -0.0644,  0.9715],
        [ 0.1131,  0.0359, -0.0237,  0.9927],
        [ 0.0170,  0.0664, -0.0058,  0.9976]], dtype=torch.float64)
    """
    final_position = forward_kinematics(
        root_joint_pos = primary_positions_root,
        all_joint_offset = root_to_end_offset,
        rotation_param_matrix = rotation_matrix
    )
    x = torch.from_numpy(np.array(final_position) 
                         - np.array(target_pos)).requires_grad_(True)
    y = 0.5 * torch.dot(x, x)
    
    optimizer = torch.optim.SGD([rotation_matrix], lr = learning_rate)
    for epoch in range(epochs):
        optimizer.zero_grad()
        final_position = forward_kinematics(
            root_joint_pos = primary_positions_root,
            all_joint_offset = root_to_end_offset,
            rotation_param_matrix = rotation_matrix
        )
        # x = torch.tensor(final_position, dtype=torch.float64) \
        #     - torch.tensor(target_pos, dtype=torch.float64)
        # y = 0.5 * torch.dot(x, x)
        final_position_tensor = torch.tensor(final_position, dtype=torch.float64, requires_grad=True)
        target_pos_tensor = torch.tensor(target_pos, dtype=torch.float64)
        x = final_position_tensor - target_pos_tensor
        y = 0.5 * torch.dot(x, x)
        y.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f"epoch {epoch} loss {y.item()}")
        
        if y.item() < thresold:
            break
        
    final_joint_positions = forward_kinematics(
        root_joint_pos = primary_positions_root,
        all_joint_offset = root_to_end_offset,
        rotation_param_matrix = rotation_matrix
    )
    final_orientation = rotation_matrix.detach().numpy()
    
    return final_joint_positions, final_orientation
        


In [17]:
bone_tree_T_pos = Bone_Tree("walk60.bvh")
path_info = bone_tree_T_pos.get_path_from_Ajoint_To_Bjoint("lToeJoint_end", "RootJoint")
target_pos = [0.35, 0.45, 0.13]


In [25]:
final_joint_positions, final_orientation = inverse_kinematics(
    bone_tree = bone_tree_T_pos,
    path_info = path_info,
    target_pos = target_pos
)

epoch 0 loss 1.0387181264466132
epoch 10 loss 1.0387181264466132
epoch 20 loss 1.0387181264466132
epoch 30 loss 1.0387181264466132
epoch 40 loss 1.0387181264466132
epoch 50 loss 1.0387181264466132
epoch 60 loss 1.0387181264466132
epoch 70 loss 1.0387181264466132
epoch 80 loss 1.0387181264466132
epoch 90 loss 1.0387181264466132
epoch 100 loss 1.0387181264466132
epoch 110 loss 1.0387181264466132
epoch 120 loss 1.0387181264466132
epoch 130 loss 1.0387181264466132
epoch 140 loss 1.0387181264466132
epoch 150 loss 1.0387181264466132
epoch 160 loss 1.0387181264466132
epoch 170 loss 1.0387181264466132
epoch 180 loss 1.0387181264466132
epoch 190 loss 1.0387181264466132
epoch 200 loss 1.0387181264466132
epoch 210 loss 1.0387181264466132
epoch 220 loss 1.0387181264466132
epoch 230 loss 1.0387181264466132
epoch 240 loss 1.0387181264466132
epoch 250 loss 1.0387181264466132
epoch 260 loss 1.0387181264466132
epoch 270 loss 1.0387181264466132
epoch 280 loss 1.0387181264466132
epoch 290 loss 1.03871812

In [19]:
print(final_joint_positions)
print(final_orientation)

[-0.03453334526386108, -0.7794460994715716, -0.5165544430034866]
[[ 0.017026    0.06643844 -0.00578745  0.99762847]
 [ 0.1130767   0.03588904 -0.02372451  0.99265441]
 [ 0.22701368  0.02261524 -0.0644312   0.9714947 ]
 [ 0.07547421 -0.02394533  0.00391293  0.99685252]
 [ 0.00652976 -0.02040147 -0.01068351  0.99971346]
 [ 0.00652976 -0.02040147 -0.01068351  0.99971346]]
